# Data Visualization

In [ ]:
import pandas as pd
from visualization import plot_on_map

labels = pd.read_csv("datasets/xbd/tier3_bldgs/joplin-tornado/joplin-tornado_tier3_labels.csv", index_col=0)
plot_on_map(labels, color='class')

In [ ]:
labels = pd.read_csv("datasets/xbd/tier3_bldgs/nepal-flooding/nepal-flooding_tier3_labels.csv", index_col=0)
plot_on_map(labels)

In [ ]:
labels = pd.read_csv("datasets/xbd/tier3_bldgs/lower-puna-volcano/lower-puna-volcano_tier3_labels.csv", index_col=0)
plot_on_map(labels)

## Polygon Visualization

In [ ]:
from visualization import plot_image

plot_image("datasets/xbd/tier3/labels/joplin-tornado_00000002_post_disaster.json")

---

In [ ]:
import numpy as np
from train_autoencoder import _make_cost_m
from scipy.optimize import linear_sum_assignment

cm = np.array([[46,26,475,26,11],[5,445,31,108,9],[4,506,32,54,1],[164,31,43,158,197],[408,12,6,15,123]])
cm

In [ ]:
indexes = linear_sum_assignment(_make_cost_m(cm))
indexes

In [ ]:
cm2 = cm[:, indexes[1]]
cm2

In [ ]:
#FP
cm2.sum(axis=0) - np.diag(cm2)

In [ ]:
#FN
cm2.sum(axis=1) - np.diag(cm2)

# Beirut Images

In [ ]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt

In [ ]:
gdf = gpd.read_file("D:/ArcGIS/BeirutPort/buffered_masks.shp")[['damage_num', 'Longitude', 'Latitude', 'geometry']]
gdf.head()

In [ ]:
preimg = rasterio.open("D:/ArcGIS/BeirutPort/PreExp.tif")
postimg = rasterio.open("D:/ArcGIS/BeirutPort/PostExp.tif")

In [ ]:
i = 971#463
out_img, _ = mask(preimg, shapes=[gdf['geometry'][i]], crop=True)
plt.figure()
plt.imshow(np.moveaxis(out_img,0,-1))
plt.axis('off')
plt.show()
out_img, _ = mask(postimg, shapes=[gdf['geometry'][i]], crop=True)
plt.figure()
plt.imshow(np.moveaxis(out_img,0,-1))
plt.axis('off')
plt.show()